<a href="https://colab.research.google.com/github/mathluva/TwitterSentiment-BERT/blob/main/SentimentTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
!nvidia-smi

Sun Feb 28 16:51:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W /  70W |   1528MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [68]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [69]:
from google.colab import files
uploaded = files.upload()

Saving smile-annotations-final.csv to smile-annotations-final (2).csv


In [70]:
import io
df = pd.read_csv(io.BytesIO(uploaded['smile-annotations-final.csv']))
df.head()

,611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap,nocode
0,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
1,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
2,614877582664835073,@Sofabsports thank you for following me back. ...,happy
3,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
4,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy


In [71]:
df = df.rename(columns={'611857364396965889': 'id','@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap': 'text', 'nocode':'category'})
df.head()

,id,text,category
0,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
1,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
2,614877582664835073,@Sofabsports thank you for following me back. ...,happy
3,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
4,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy


In [72]:
rows = len(df)
print(f'There are {rows} rows')

There are 3084 rows


In [73]:
df.tail(20)

,id,text,category
3064,610575620641300480,Big up to #kayaktivists @cmkshama #Seattle #S...,nocode
3065,614089268198285312,Fantastic news for #digitalpreservation as @dr...,happy
3066,611537640857411584,@britishmuseum @SenderosP The Rosetta Stone ;),happy
3067,611227963976253440,THIS is why #ilovemuseums ! Torres Strait danc...,happy
3068,614074786042486785,I wonder what hitherto unknown treasures might...,nocode
3069,612242969035411456,@britishmuseum on INSTA. See this magnificent ...,happy
3070,614900716960915456,This was definitely one of the best cocktails ...,happy
3071,611557802197299201,James Gillray (1756-1815) @britishmuseum until...,nocode
3072,611532293358899200,@DerekSharplol @britishmuseum How about Beef W...,nocode
3073,613646155591909377,#Caravaggio painted the beheading of St John t...,nocode


In [74]:
#replace index with id
df.set_index('id', inplace =True) 

In [75]:
#check sum of identity_hate
df.category.value_counts()

nocode               1571
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [76]:
df = df[~df.category.str.contains('\|')]
df = df[df.category != 'nocode']

In [77]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [78]:
#create new column to hold label value
possible_labels = df.category.unique()

label_dict = {}
for index, label in enumerate(possible_labels):
    label_dict[label] = index

In [79]:
df['label'] = df.category.replace(label_dict)

In [80]:
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


In [81]:
#check totals
df['label'].value_counts()

0    1137
1     214
2      57
5      35
4      32
3       6
Name: label, dtype: int64

In [83]:
#use stratified sampling to ensure that sampled dataset is representative of the entire population
X_train, X_val, y_train, y_val= train_test_split(df.index.values, df.label.values, random_state = 42, stratify = df['label'].values)

In [84]:
#create set column
#locate training and val samples
df['data_type'] = ['none']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [85]:
#check stratified sampling values
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        43
                   val          14
disgust      3     train         5
                   val           1
happy        0     train       852
                   val         285
not-relevant 1     train       160
                   val          54
sad          4     train        24
                   val           8
surprise     5     train        26
                   val           9

In [86]:
!pip install transformers==3.5

In [87]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [88]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)


In [89]:
#encode train data
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    padding = True,
    truncation = True,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)



In [90]:
#encode validation data
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
      padding = True,
    truncation = True,
    return_tensors='pt'
)


input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [91]:
# Convert all inputs and labels into torch tensors
#BERT model required datatype 
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [92]:
print(len(dataset_train))
print(len(dataset_val))

1110
371


In [93]:
from transformers import BertForSequenceClassification

In [94]:
 model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                       num_labels=len(possible_labels),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [95]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [96]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [97]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [101]:
#https://huggingface.co/transformers/main_classes/optimizer_schedules.html

optimizer = AdamW(
    model.parameters(),
    lr=1e-3
)

In [102]:
#Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0,
#after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

epochs = 10
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [103]:
from sklearn.metrics import f1_score

In [104]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [105]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [106]:
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [107]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [108]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [110]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train( )
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.0276563227176667
Validation loss: 0.8147139996290207
F1 Score (Weighted): 0.6674857011373347



Epoch 2
Training loss: 0.8399325609207153
Validation loss: 0.8160218497117361
F1 Score (Weighted): 0.6674857011373347



Epoch 3
Training loss: 0.8493734478950501
Validation loss: 0.8213922083377838
F1 Score (Weighted): 0.6674857011373347



Epoch 4
Training loss: 0.8354639998504094
Validation loss: 0.820921391248703
F1 Score (Weighted): 0.6674857011373347



Epoch 5
Training loss: 0.8389478487627847
Validation loss: 0.8258327146371206
F1 Score (Weighted): 0.6674857011373347



Epoch 6
Training loss: 0.8357078654425485
Validation loss: 0.8059844772020975
F1 Score (Weighted): 0.6674857011373347



Epoch 7
Training loss: 0.8328462328229632
Validation loss: 0.8005099793275198
F1 Score (Weighted): 0.6674857011373347



Epoch 8
Training loss: 0.8123129257134029
Validation loss: 0.8213781615098318
F1 Score (Weighted): 0.6674857011373347



Epoch 9
Training loss: 0.8220411649772099
Validation loss: 0.798029214143753
F1 Score (Weighted): 0.6674857011373347



Epoch 10
Training loss: 0.8116044495786939
Validation loss: 0.7968872437874476
F1 Score (Weighted): 0.6674857011373347



In [111]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [112]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 285/285

Class: not-relevant
Accuracy: 0/54

Class: angry
Accuracy: 0/14

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 0/8

Class: surprise
Accuracy: 0/9



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
model_save_name = 'classifierII.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)